In [1]:
import cv2
import os
import scipy.spatial.distance as dist
from threading import Thread
import numpy as np
import dlib
import time
import argparse
from pygame import mixer

pygame 1.9.4
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:

def eye_aspect_ratio(eye):
    
    v1=dist.euclidean(eye[1],eye[5])
    v2=dist.euclidean(eye[2],eye[4])
    h1=dist.euclidean(eye[0],eye[3])
    
    
    ratio=v1+v2/(2.0*h1)
    return ratio

def play_alarm():
    
    mixer.init()
    mixer.music.load('Industrial Alarm-SoundBible.com-1012301296.wav')
    mixer.music.play()
    mixer.stop()

In [ ]:
'''ap=argparse.ArgumentParser()
ap.add_argument("-p","--shape_predcitor",required=True,help="Path to Facial Landmark Detector")
args=vars(ap.parse_args())'''


In [23]:
EYE_AR_THRESHOLD=8
CONS_FRAME_BLINK=4
TOTAL=0
COUNTER=0
ALARM_ON=False

In [11]:
detector=dlib.get_frontal_face_detector()
predictor = dlib.shape_predictor("shape_predictor_68_face_landmarks.dat")

In [12]:
FACIAL_LANDMARKS_68_INDEX = {"mouth":(48, 68), "inner_mouth":(60, 68) ,
                            "right_eyebrow": (17, 22),"left_eyebrow": (22, 27),
                             "right_eye": (36, 42),"left_eye": (42, 48),
                             "nose": (27, 36),"jaw": (0, 17)
                            }

def convert_to_numpy(shape):
    
    facial_landmarks= np.zeros((68, 2), dtype=int)
    for i in range(0,shape.num_parts):
        facial_landmarks[i]=(shape.part(i).x,shape.part(i).y)
        
    return facial_landmarks

def rect_to_bounding_box(rect):
    
    x=rect.left()
    y=rect.top()
    
    w=rect.right()-x
    h=rect.bottom()-y

    return (x,y,w,h)

In [24]:
cap=cv2.VideoCapture(0)  #argument can be a device index or name/path of the video file 

cap.set(3,780)
cap.set(4,780)
#print "hello"
while(cap.isOpened()):
    
    ret,frame=cap.read() # ret=True if a frame is read ,otherwise False
    
    if ret==True:
        
        #gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY) # Convert the frame into gray image
        rects=detector(frame,0)
        
        for (i,rect) in enumerate(rects):
            
            shape = predictor(frame, rect)
            shape = convert_to_numpy(shape)
     
            #(x,y,w,h)=rect_to_bounding_box(rect)
            #cv2.rectangle(frame,(x,y),(x+w,y+h),(0,255,0),2)

            left_eye = shape[FACIAL_LANDMARKS_68_INDEX["left_eye"][0]:FACIAL_LANDMARKS_68_INDEX["left_eye"][1]]
            right_eye = shape[FACIAL_LANDMARKS_68_INDEX["right_eye"][0]:FACIAL_LANDMARKS_68_INDEX["right_eye"][1]]
            mouth = shape[FACIAL_LANDMARKS_68_INDEX["mouth"][0]:FACIAL_LANDMARKS_68_INDEX["mouth"][1]]
            nose = shape[FACIAL_LANDMARKS_68_INDEX["nose"][0]:FACIAL_LANDMARKS_68_INDEX["nose"][1]]
            inner_mouth = shape[FACIAL_LANDMARKS_68_INDEX["inner_mouth"][0]:FACIAL_LANDMARKS_68_INDEX["inner_mouth"][1]]
            right_eyebrow = shape[FACIAL_LANDMARKS_68_INDEX["right_eyebrow"][0]:FACIAL_LANDMARKS_68_INDEX["right_eyebrow"][1]]
            left_eyebrow = shape[FACIAL_LANDMARKS_68_INDEX["left_eyebrow"][0]:FACIAL_LANDMARKS_68_INDEX["left_eyebrow"][1]]
            #jaw = shape[FACIAL_LANDMARKS_68_INDEX["jaw"][0]:FACIAL_LANDMARKS_68_INDEX["jaw"][1]]
            
            left_ratio = eye_aspect_ratio(left_eye)
            right_ratio = eye_aspect_ratio(right_eye)
 

            aspect_ratio = (left_ratio + right_ratio) / 2.0
    
            left_convex_hull=cv2.convexHull(left_eye)
            right_convex_hull=cv2.convexHull(right_eye)
            
            #left_eyebrow=cv2.convexHull(left_eyebrow)
            #right_eyebrow=cv2.convexHull(right_eyebrow)
            #mouth=cv2.convexHull(mouth)
            #inner_mouth=cv2.convexHull(inner_mouth)
            #jaw=cv2.convexHull(jaw)
            #nose=cv2.convexHull(nose)
            
            cv2.drawContours(frame, [left_convex_hull], -1, (255, 0, 0), 1)
            cv2.drawContours(frame, [right_convex_hull], -1, ( 255, 0,0), 1)
            '''
            cv2.drawContours(frame, [left_eyebrow], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [right_eyebrow], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [mouth], -1, (0, 255, 0), 1)
            cv2.drawContours(frame, [inner_mouth], -1, (0, 255, 0), 1)
            #cv2.drawContours(frame, [jaw], -1, ( 255, 0,0), 1)
            cv2.drawContours(frame, [nose], -1, ( 255, 0,0), 1)
            
            '''
            if aspect_ratio < EYE_AR_THRESHOLD:
                print(aspect_ratio)
                COUNTER+=1
            
                if COUNTER>=CONS_FRAME_BLINK:
                    print("hi")
                    TOTAL+=1
                    
                    if ALARM_ON==False:
                        ALARM_ON=True
                        alarm_thread=Thread(target=play_alarm)
                        alarm_thread.deamon=True
                        alarm_thread.start()
            else:
                COUNTER=0
                ALARM_ON=False
            
            cv2.putText(frame, "Blinks: {}".format(TOTAL), (10, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 2)
            cv2.putText(frame, "EAR: {:.2f}".format(aspect_ratio), (300, 30),
                        cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 220, 255), 1)
            
            cv2.imshow('frame',frame)
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break
    else:
        break

# When everything done, release the capture
cap.release()
cv2.destroyAllWindows()

7.11747618189
7.66182644265
7.62415679216
7.13366732263
7.09907008617
hi
6.58696306546
hi
7.63541021863
hi
7.59973790103
hi
6.63207822681
hi
7.0953794221
hi
7.0997145041
hi
7.08696306546
hi
7.59650408324
hi


In [ ]:
ar=(10,20)
ar[1]